# Effect size Calculation
We have developed a Python-based example for invoking R to compute effect sizes. Please ensure that the code is executed on your Linux server with R properly installed and configured.

## Dichotomous Outcome


In [ ]:
import pandas as pd

# Construct 2×2 data: ai, bi, ci, di
# For example, each row represents a study with event.e / total.e and event.c / total.c
data = {
    'study': [
        'study1',
        'study2',
        'study3',
        'study4',
        # 'study5',
        # 'study6',
        # 'study7', you can add more studies if needed
              ],
    'ai': [25,10,83,324],   # Number of events in the experimental group
    'bi': [17,9,97,174],    # Number of non-events in the experimental group
    'ci': [29,8,95,256],    # Number of events in the control group
    'di': [14,8,87,245],    # Number of non-events in the control group
}
df = pd.DataFrame(data)
df

In [96]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()  # Activate pandas <-> R data.frame conversion

r_df = pandas2ri.py2rpy(df)    
ro.globalenv['meta_data'] = r_df  # In R, you can access it using meta_data


In [ ]:
# Load the meta package
ro.r('library(meta)')

# Use metabin() with measure="RR" (or "OR") and model="R" for random effects model
ro.r('res <- metabin(event.e=ai, n.e=ai+bi, event.c=ci, n.c=ci+di, data=meta_data, sm="RR", method="MH", fixed=FALSE, random=TRUE, studlab=study)')
# Capture the output of summary(res) in R
captured = ro.r('capture.output(summary(res))')
# captured is a Python list, each element is a line
for line in captured:
    print(line)


In [ ]:
# Absolute effect (RD) analysis, i.e., Risk Difference (Absolute Effect)
ro.r('res_rd <- metabin(event.e=ai, n.e=ai+bi, event.c=ci, n.c=ci+di, '
     'data=meta_data, sm="RD", method="MH", fixed=FALSE, random=TRUE,studlab=study)')

captured_rd = ro.r('capture.output(summary(res_rd))')
print("\n=== Absolute Effect (RD) ===")
for line in captured_rd:
    print(line)

### Optional

In [ ]:
from IPython.display import Image

# Use forest() to draw a forest plot and save it as a PNG
ro.r('''
png("forest_RR.png", width=1000, height=600, res=100)
forest(res, main="Risk Ratio (Random Effects)", comb.random=TRUE, comb.fixed=FALSE)
dev.off()
''')
Image("forest_RR.png")


In [ ]:
# Draw a funnel plot and save it as a PNG
# For the RR analysis result (res), generate a funnel plot
ro.r('''
png("funnel_RR.png", width=800, height=600, res=120)
funnel(res, main="Funnel Plot (RR)",
       xlab="Log Risk Ratio", ylab="Standard Error of Log(RR)")
dev.off()
''')
Image("funnel_RR.png")

## Continuous Outcome

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd

# 1. Assume you already have a pandas DataFrame with the following column names:
# study, mean_t, sd_t, n_t, mean_c, sd_c, n_c
# These represent: study name, treatment group mean, treatment group SD, treatment group sample size,
# control group mean, control group SD, control group sample size

data = {
    'study': ['Study 1'],
    'mean_t': [4.4],    # Mean of the treatment group
    'sd_t': [1.4],      # Standard deviation of the treatment group
    'n_t': [49],        # Sample size of the treatment group
    'mean_c': [5.2],    # Mean of the control group
    'sd_c': [1.5],      # Standard deviation of the control group
    'n_c': [53]         # Sample size of the control group
}

df = pd.DataFrame(data)

# 2. Enable pandas <-> R conversion
pandas2ri.activate()

# 3. Pass the DataFrame to R and name it meta_data
r_df = pandas2ri.py2rpy(df)
ro.globalenv['meta_data'] = r_df

# 4. Call R code
# Load the meta package
ro.r('library(meta)')

# Perform meta-analysis for continuous outcomes using metacont()
# sm="MD" indicates that Mean Difference will be calculated
ro.r('res <- metacont(n.e = n_t, mean.e = mean_t, sd.e = sd_t, '
     'n.c = n_c, mean.c = mean_c, sd.c = sd_c, '
     'data = meta_data, sm = "MD", method.tau = "REML", '
     'fixed = FALSE, random = TRUE, studlab = study)')

# 5. Capture the output of summary(res)
captured = ro.r('capture.output(summary(res))')

# 6. Print the results
for line in captured:
    print(line)
